# Time Series Analysis of SEDAC PM2.5 Grids via Statistical API

In [1]:
import os
import yaml
from munch import munchify

### _Setup configuration and environment_

In [2]:
# define repo name and get root working directory
repo = 'statistical-api'
root_path = os.getcwd()[ 0 : os.getcwd().find( repo ) + len ( repo )]

In [3]:
# get path to configuration files
cfg_path = os.path.join( root_path, 'cfg' )
cfg_path = os.path.join( cfg_path, 'byoc' )
cfg_path

'C:\\Users\\crwil\\Documents\\GitHub\\sentinelhub\\statistical-api\\cfg\\byoc'

In [4]:
# get pathname to configuration file
cfg_file = os.path.join( cfg_path, 'pm25.yml' )

In [5]:
# print contents of configuration file
with open( cfg_file ) as f:
    print ( f.read() )

request:
    evalscript: |
                //VERSION=3
                function setup() {
                    return {
                        input: [{
                            bands: [
                                "pm25",
                                "dataMask"
                            ]
                        }],
                        output: [
                        {
                            id: "output_pm25",
                            bands: 1,
                            sampleType: "FLOAT32"
                        },
                        {
                            id: "dataMask",
                            bands: 1
                        } ]
                    }
                }
                function evaluatePixel(samples) {
                    return {
                        output_pm25: [samples.pm25],
                        dataMask: [samples.dataMask]
                    }
                }    
    inputs:
        - collection: byoc-8264

In [6]:
# load cfg file using yaml parser
with open( cfg_file, 'r' ) as f:
    config = munchify( yaml.safe_load( f ) )
config

Munch({'request': Munch({'evalscript': '//VERSION=3\nfunction setup() {\n    return {\n        input: [{\n            bands: [\n                "pm25",\n                "dataMask"\n            ]\n        }],\n        output: [\n        {\n            id: "output_pm25",\n            bands: 1,\n            sampleType: "FLOAT32"\n        },\n        {\n            id: "dataMask",\n            bands: 1\n        } ]\n    }\n}\nfunction evaluatePixel(samples) {\n    return {\n        output_pm25: [samples.pm25],\n        dataMask: [samples.dataMask]\n    }\n}    \n', 'inputs': [Munch({'collection': 'byoc-8264570a-02b7-4196-ae8e-0c9d7af766e7'})]}), 'responses': None})

### _Replicate use case results taken from Sentinel-Hub documentation_

In [7]:
# define aggregation timeframe
from datetime import datetime
timeframe = { 'start' : datetime.strptime('2000-01-01', '%Y-%m-%d'), 
              'end' : datetime.strptime('2021-01-01', '%Y-%m-%d') }

In [8]:
# create instance of shclient class
from statisticalapi import Client
client = Client( config )

In [9]:
# define min and max latlons
coords = -5, 51, -1, 53
resolution = 1000

bbox = client.getBoundingBox( coords )

In [10]:
# submit request
response = client.getStatistics( [ timeframe ], resolution=resolution, bbox=bbox, interval='P1Y' )

In [11]:
response._dfs[ 0 ]

,interval_from,interval_to,output_pm25_B0_min,output_pm25_B0_max,output_pm25_B0_mean,output_pm25_B0_stDev,output_pm25_B0_sampleCount,output_pm25_B0_noDataCount
0,2000-01-01,2001-01-01,7.6,18.299999,12.249942,1.739621,61050,13341
1,2001-01-01,2002-01-01,8.2,19.200001,12.790654,1.803151,61050,13341
2,2002-01-01,2003-01-01,10.8,23.500000,17.940594,2.467686,61050,13341
3,2003-01-01,2004-01-01,6.7,17.500000,11.417408,2.019724,61050,13341
4,2004-01-01,2005-01-01,8.0,17.799999,11.906538,1.599670,61050,13341
5,2005-01-01,2006-01-01,6.9,16.000000,10.671603,1.883374,61050,13341
6,2006-01-01,2007-01-01,7.8,16.900000,11.862552,1.460609,61050,13341
7,2007-01-01,2008-01-01,7.6,18.000000,12.353447,2.292853,61050,13341
8,2008-01-01,2009-01-01,6.6,15.400000,10.642160,1.672791,61050,13341
9,2009-01-01,2010-01-01,6.7,14.500000,10.253164,1.400760,61050,13341
